In [1]:
import pandas as pd
import numpy as np
import pickle
import os

os.chdir('/Users/a.kholodov/Documents/02. Personal/20. Education/50. Universities/Springboard/Springboard_git/Springboard_repo/CS2-flights-delay-REPO')


input_file_name = 'data/processed/processed_2014_2018_pre_final.pickle'
with open(input_file_name, 'rb') as in_file:
    flights = pickle.load(in_file)

In [2]:
def load_data_from(zip_file, data_file, field_type=None):
    '''
    Description
    -----------
    Load data specified as field_type from one data file from zip-archive 

    Parameters
    -----------
    zip_file - path and name of source zip-file contaning 60 csv files
    dat_faile - path and name of csv-file with data
    field_type - dictinary with fields to load and thiers relative data types

    Returns
    -----------
    DataFrame with data loaded
    '''

    # reading the file
    with zipfile.ZipFile(zip_file) as zip_source:
        with zip_source.open(data_file) as file:
            if field_type != None:
                df = pd.read_csv(file, header = 0, 
                                usecols = field_type.keys(),
                                dtype = field_type)
            else:
                df = pd.read_csv(file, header = 0, low_memory=False)

    # Converting dates and boolean        
    if 'FlightDate' in df.columns:
        df['FlightDate'] = pd.to_datetime(df['FlightDate'])
    if 'DivReachedDest' in df.columns:
        df['DivReachedDest'] = df['DivReachedDest'].fillna(0)
    if 'Cancelled' in df.columns:
        df['Cancelled'] = df['Cancelled'].astype('bool')
    if 'Diverted' in df.columns:
        df['Diverted'] = df['Diverted'].astype('bool')
    if 'DivReachedDest' in df.columns:
        df['DivReachedDest'] = df['DivReachedDest'].astype('bool')
    return df

In [3]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29871484 entries, 0 to 30144614
Data columns (total 29 columns):
 #   Column                           Dtype                  
---  ------                           -----                  
 0   Reporting_Airline                category               
 1   Flight_Number_Reporting_Airline  int16                  
 2   Origin                           category               
 3   Dest                             category               
 4   DepartureDelayGroups             category               
 5   DepTimeBlk                       category               
 6   ArrivalDelayGroups               category               
 7   ArrTimeBlk                       category               
 8   Cancelled                        bool                   
 9   CancellationCode                 category               
 10  Diverted                         bool                   
 11  AirTime                          float16                
 12  CarrierDelay     

In [4]:
pd.pivot_table(data = flights, index = ['Cancelled', 'Diverted'], columns=['DivReachedDest'], values = 'CRSDepDT', aggfunc="count", fill_value='', margins=True)

/var/folders/_s/q9nyv1gx3_70t9n320c0cb4h0000gn/T/ipykernel_64163/239027940.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pd.pivot_table(data = flights, index = ['Cancelled', 'Diverted'], columns=['DivReachedDest'], values = 'CRSDepDT', aggfunc="count", fill_value='', margins=True)


DivReachedDest         False   True       All
Cancelled Diverted                           
False     False     29316066         29316066
          True         11823  61619     73442
True      False       481961           481961
          True            15               15
All                 29809865  61619  29871484

In [ ]:
# t = (~flights['Cancelled']) & (~flights['Diverted'] | flights['DivReachedDest'])
actually_arrived_flights = (~flights['Cancelled'] & ~(flights['Diverted'] & ~flights['DivReachedDest']))

In [ ]:
actually_arrived_flights.sum()

29377685

In [ ]:
arrived_na_delay = flights.loc[actually_arrived_flights & flights['ActArrDelay'].isna(), ['Reporting_Airline', 'Flight_Number_Reporting_Airline', 'Origin', 'Dest', 'CRSDepDT', 
                                                          'Cancelled', 'Diverted', 'DivReachedDest', 'ActArrDelay']]

with pd.option_context('display.max_rows', 300, 'display.width', 200):
    print(arrived_na_delay)
print(len(arrived_na_delay))

         Reporting_Airline  Flight_Number_Reporting_Airline Origin Dest            CRSDepDT  Cancelled  Diverted  DivReachedDest  ActArrDelay
35061                   DL                             1641    SAT  ATL 2014-01-06 11:02:00      False      True            True          NaN
242000                  MQ                             3588    MSP  LGA 2014-01-05 17:00:00      False      True            True          NaN
354545                  AA                               24    SFO  JFK 2014-01-05 07:05:00      False      True            True          NaN
1050066                 DL                             2116    OGG  LAX 2014-03-12 22:00:00      False      True            True          NaN
1228082                 HA                                7    LAS  HNL 2014-03-24 09:05:00      False      True            True          NaN
1858790                 HA                               46    OGG  SJC 2014-04-18 12:30:00      False      True            True          NaN
189567

In [ ]:
flights.loc[actually_arrived_flights & flights['ActArrDelay'].isna()].index.tolist()



[35061,
 242000,
 354545,
 1050066,
 1228082,
 1858790,
 1895678,
 1897261,
 1909068,
 1922076,
 1928989,
 2396490,
 2405487,
 2900628,
 2913967,
 2923126,
 2987023,
 3415877,
 3420129,
 3891769,
 4406893,
 4902644,
 6612655,
 6621448,
 6622227,
 6629496,
 6632712,
 6633066,
 6827573,
 6827750,
 6845038,
 6859675,
 7468029,
 7469737,
 7472979,
 7485282,
 7495253,
 8092596,
 8110997,
 9618671,
 9728388,
 10233246,
 10239400,
 11162229,
 11168179,
 11180258,
 11183537,
 11184089,
 11184758,
 11191380,
 11191636,
 11192750,
 11195434,
 11196071,
 11278058,
 11278612,
 11643386,
 11643593,
 12086461,
 12099920,
 12660901,
 12671807,
 13469769,
 13955626,
 13967855,
 14177994,
 14416434,
 14418200,
 14436663,
 14671354,
 14944246,
 14959110,
 15326921,
 15876282,
 16348102,
 16436498,
 16679557,
 16797440,
 16858085,
 17265080,
 17283977,
 17397114,
 17496956,
 17590101,
 17601135,
 17822441,
 18124285,
 18429240,
 18436430,
 18441211,
 18443972,
 18632942,
 18633856,
 18948325,
 18982010,


In [28]:
flight_num = flights['Flight_Number_Reporting_Airline'] == 1641
airline = flights['Reporting_Airline'] == 'DL'
origin = flights['Origin'] == 'SAT'
dest = flights['Dest'] == 'ATL'
date = flights['CRSDepDT'] == '2014-01-06 11:02:00'

filter = flight_num & airline & origin & dest & date
with pd.option_context('display.max_rows', 112, 'display.width', 500):
    print(flights.loc[filter].T)

                                                     35061
Reporting_Airline                                       DL
Flight_Number_Reporting_Airline                       1641
Origin                                                 SAT
Dest                                                   ATL
DepartureDelayGroups                                     0
DepTimeBlk                                       1100-1159
ArrivalDelayGroups                                     NaN
ArrTimeBlk                                       1400-1459
Cancelled                                            False
CancellationCode                                       NaN
Diverted                                              True
AirTime                                                NaN
CarrierDelay                                           NaN
WeatherDelay                                           NaN
NASDelay                                               NaN
SecurityDelay                                          N

In [9]:
import zipfile

source_zip_file = 'data/interim/csv_flight.zip'
data_file = 'csv_flight/report_2017_4.csv'

# reading the first file to evaluate the data
flights_2017_4 = load_data_from(source_zip_file, data_file)

In [10]:
# F9                              920    MCO  TTN 2017-04-04 20:35:00

flight_num = flights_2017_4['Flight_Number_Reporting_Airline'] == 920
airline = flights_2017_4['Reporting_Airline'] == 'F9'
origin = flights_2017_4['Origin'] == 'MCO'
dest = flights_2017_4['Dest'] == 'TTN'
date = flights_2017_4['FlightDate'] == '2017-04-04'

filter = flight_num & airline & origin & dest & date
with pd.option_context('display.max_rows', 112):
    print(flights_2017_4.loc[filter].T)

                                              342646
Year                                            2017
Quarter                                            2
Month                                              4
DayofMonth                                         4
DayOfWeek                                          2
FlightDate                       2017-04-04 00:00:00
Reporting_Airline                                 F9
DOT_ID_Reporting_Airline                       20436
IATA_CODE_Reporting_Airline                       F9
Tail_Number                                   N214FR
Flight_Number_Reporting_Airline                  920
OriginAirportID                                13204
OriginAirportSeqID                           1320402
OriginCityMarketID                             31454
Origin                                           MCO
OriginCityName                           Orlando, FL
OriginState                                       FL
OriginStateFips                               

In [46]:
input_file_name = 'data/processed/processed_2014_1.pickle'
print(input_file_name)
with open(input_file_name, 'rb') as in_file:
    f_pckl = (pickle.load(in_file))

data/processed/processed_2014_1.pickle


In [47]:
flight_num = f_pckl['Flight_Number_Reporting_Airline'] == 1641
airline = f_pckl['Reporting_Airline'] == 'DL'
origin = f_pckl['Origin'] == 'SAT'
dest = f_pckl['Dest'] == 'ATL'
date = f_pckl['CRSDepDT'] == '2014-01-06 11:02:00'

filter = flight_num & airline & origin & dest & date
with pd.option_context('display.max_rows', 112, 'display.width', 500):
    print(f_pckl.loc[filter].T)

                                                     35061
Reporting_Airline                                       DL
Flight_Number_Reporting_Airline                       1641
Origin                                                 SAT
Dest                                                   ATL
DepDelay                                              10.0
DepartureDelayGroups                                     0
DepTimeBlk                                       1100-1159
ArrDelay                                               NaN
ArrivalDelayGroups                                     NaN
ArrTimeBlk                                       1400-1459
Cancelled                                            False
CancellationCode                                       NaN
Diverted                                              True
CRSElapsedTime                                       133.0
ActualElapsedTime                                   1403.0
AirTime                                                N